# Very naive learning on the vector of surrounding bases with xgboost

---


### Data

Naive feature vectors. The original sequence of validation/test and train data does not overlap! ( but train data points can overlap with train data points, and test-validation can overlap with test-validation data ) This overlapping does not lead to unintentional label leakage!



### Analysis

I will try to naively predict the mehtylation value (0,1) based on naive sequence context feauture vectors.

I will use xgboost, a sophisticated library for gradient boosting, which is used to win many kaggle competitions.
    



### Conlcusions

---

In [1]:
import numpy as np
import time
import xgboost as xgb

import sys
sys.path.append('../my_modules/')
from loading_utils import read_my_data,create_sets

import os,subprocess
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

### Load data

In [2]:
train_x,train_y = read_my_data(fname='../prepare_data/big_train_feat_vect.csv')
valid_x,valid_y = read_my_data(fname='../prepare_data/big_val_feat_vect.csv')
test_x,test_y = read_my_data(fname='../prepare_data/big_test_feat_vect.csv')

Loading data... 
Loading data... 
Loading data... 


#### Create xgboost matrices

In [3]:
dtrain = xgb.DMatrix( train_x, label=train_y)
dvalid = xgb.DMatrix( valid_x, label=valid_y)
dtest = xgb.DMatrix( test_x, label=test_y)

### With modest overfitting

In [6]:
param = {'max_depth':4,
         'eta':0.1,
         'min_child_weight':50,
         'colsample_bytree':0.8,
         'subsample':0.6,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':8}

evallist  = [(dtrain,'train'),(dvalid,'eval')]

#train
num_round = 2000
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round,
                early_stopping_rounds=100,
                verbose_eval=50)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-error:0.416017	eval-error:0.418700
[50]	train-error:0.244564	eval-error:0.258850
[100]	train-error:0.228000	eval-error:0.246650
[150]	train-error:0.220255	eval-error:0.239300
[200]	train-error:0.214027	eval-error:0.235750
[250]	train-error:0.207472	eval-error:0.231900
[300]	train-error:0.200959	eval-error:0.229350
[350]	train-error:0.194681	eval-error:0.226050
[400]	train-error:0.187773	eval-error:0.221900
[450]	train-error:0.181763	eval-error:0.219450
[500]	train-error:0.175074	eval-error:0.212650
[550]	train-error:0.168980	eval-error:0.209200
[600]	train-error:0.163431	eval-error:0.205800
[650]	train-error:0.157438	eval-error:0.202850
[700]	train-error:0.151922	eval-error:0.199350
[750]	train-error:0.146331	eval-error:0.197150
[800]	train-error:0.140597	eval-error:0.194350
[850]	train-error:0.135459	eval-error:0.192700
[900]	train-error:0.131603	eval-error:0.189800
[950]	train-error:0.127563	eval-error:0.188400
[10

#### Final scores

In [7]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.92290734128
validation score: 0.8273
test score: 0.83615
